In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
movies = movies.merge(credits, on='title')

In [6]:
movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]

In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.dropna(inplace=True)

In [9]:
movies.duplicated().sum()

0

In [10]:
import json

In [11]:
def convert(obj):
    res = []
    for i in json.loads(obj):
        res.append(i['name'])
    return res

In [12]:
movies['genres'] = movies['genres'].apply(convert)

In [13]:
movies['keywords'] = movies['keywords'].apply(convert)

In [14]:
def convert3(obj):
    res = []
    counter = 0
    for i in json.loads(obj):
        counter += 1
        res.append(i['name'])
        if counter == 3:
            break
    return res

In [15]:
movies['cast'] = movies['cast'].apply(convert3)

In [16]:
def get_director(obj):
    for i in json.loads(obj):
        if i['job'] == 'Director':
            return [i['name']]
    return []

In [17]:
movies['crew'] = movies['crew'].apply(get_director)

In [18]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [19]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

In [20]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [21]:
new_df = movies[["movie_id", "title", "tags"]]

In [22]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

<ipython-input-22-e498c0e361a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [23]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

<ipython-input-23-db70cfe1859e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [31]:
import nltk

In [32]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [33]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [35]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-35-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [36]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [37]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [38]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [39]:
cv.get_feature_names()

['16thcenturi',
 '17thcenturi',
 '18thcenturi',
 '1910',
 '1930',
 '1940',
 '1950',
 '1960',
 '1970',
 '1980',
 '1990',
 '19thcenturi',
 '21stcenturi',
 '3d',
 'aaroneckhart',
 'aarontaylor',
 'abbiecornish',
 'abhishekbachchan',
 'abigailbreslin',
 'aborigin',
 'abram',
 'absurd',
 'abus',
 'abusivehusband',
 'accid',
 'action',
 'actionhero',
 'activist',
 'actor',
 'actress',
 'adambeach',
 'adambrodi',
 'adamcarolla',
 'adamdriv',
 'adamgarcia',
 'adamgoldberg',
 'adammckay',
 'adamrifkin',
 'adamsandl',
 'adamscott',
 'adamshankman',
 'adapt',
 'addict',
 'adolesc',
 'adolfhitl',
 'adopt',
 'adoptedchild',
 'adoptivefath',
 'adrianlyn',
 'adrienbrodi',
 'adriennebarbeau',
 'adultanim',
 'adulteri',
 'advanc',
 'adventur',
 'adversari',
 'advertisingexecut',
 'advertisingexpert',
 'advic',
 'affair',
 'affect',
 'afghanistan',
 'africa',
 'africanamerican',
 'aftercreditssting',
 'afterlif',
 'age',
 'agediffer',
 'agent',
 'aggressionbyanim',
 'aid',
 'aidanquinn',
 'aimeegarcia',

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
similarity = cosine_similarity(vectors)

In [47]:
similarity[0]

array([1.        , 0.12309149, 0.11605177, ..., 0.06085806, 0.        ,
       0.        ])

In [62]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]]) 
    return

In [63]:
recommend("Avatar")

movie_id                                                54138
title                                 Star Trek Into Darkness
tags        action adventur sciencefict spacecraft friends...
Name: 47, dtype: object
movie_id                                                27380
title                                               Megaforce
tags        adventur action sciencefict dictat soldier ele...
Name: 2378, dtype: object
movie_id                                                76757
title                                       Jupiter Ascending
tags        sciencefict fantasi action adventur jupit spac...
Name: 61, dtype: object
movie_id                                                79698
title                                              The Lovers
tags        action adventur sciencefict romanc joshhartnet...
Name: 83, dtype: object
movie_id                                                  679
title                                                  Aliens
tags        horror action thriller

In [58]:
import pickle

In [60]:
pickle.dump(new_df.to_dict(), open("movies_dict.pkl", "wb"))

In [61]:
pickle.dump(similarity, open('similarity_scores.pkl', 'wb'))